In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [1]:
!pip install datasets transformers tensorflow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 4.4 MB/s eta 0:00:00a 0:00:01
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.2
    Uninstalling fsspec-2025.3.2:
      Successfully uninstalled fsspec-2025.3.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
torch 2.6.0+cu124 requires nvid

In [2]:
!pip install tensorflow-model-optimization

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.5/242.5 kB 5.8 MB/s eta 0:00:00a 0:00:01


In [3]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot
import tensorflow.lite as tflite
import time
import os
import contextlib

# Step 1: Set up TPU (optional, skip for GPU/CPU)
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()
    print('Running on TPU ', tpu.cluster_spec().as_dict()['worker'])
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
except ValueError:
    strategy = None  # Use GPU/CPU if TPU not available
    print('No TPU detected, running on GPU/CPU')

2025-07-04 13:43:47.725517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751636627.896918      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751636627.948506      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


No TPU detected, running on GPU/CPU


In [5]:
# Step 2: Install required packages (if not already installed in Kaggle)
!pip install tensorflow-model-optimization
!pip install ipywidgets  # For TqdmWarning, if needed

In [6]:
# Load and preprocess the SST-2 dataset
dataset = load_dataset("glue", "sst2")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

README.md: 0.00B [00:00, ?B/s]

train-00000-of-00001.parquet:   0%|          | 0.00/3.11M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/72.8k [00:00<?, ?B/s]

test-00000-of-00001.parquet:   0%|          | 0.00/148k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/67349 [00:00<?, ? examples/s]

...

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/67349 [00:00<?, ? examples/s]

Map:   0%|          | 0/872 [00:00<?, ? examples/s]

Map:   0%|          | 0/1821 [00:00<?, ? examples/s]

In [7]:
# Prepare tf.data.Dataset (excluding token_type_ids)
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
)

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
)

/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(


In [8]:
# Step 4: Fine-tune baseline DistilBERT
with contextlib.nullcontext():
    baseline_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
    baseline_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )
    baseline_model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

# Save baseline model in SavedModel format
baseline_model.save('baseline_model', save_format='tf')

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFDistilBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['pre_classifier.weight', 'pre_classifier.bias', 'classifier.weight', 'classifier.bias']
You should 

Epoch 1/3


I0000 00:00:1751636859.550089     125 service.cc:148] XLA service 0x7b6ba4682da0 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1751636859.550877     125 service.cc:156]   StreamExecutor device (0): Tesla P100-PCIE-16GB, Compute Capability 6.0
I0000 00:00:1751636859.623604     125 cuda_dnn.cc:529] Loaded cuDNN version 90300
I0000 00:00:1751636859.767174     125 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


4210/4210 [==============================] - 575s 130ms/step - loss: 0.2185 - accuracy: 0.9149 - val_loss: 0.2462 - val_accuracy: 0.8945
Epoch 2/3
4210/4210 [==============================] - 541s 128ms/step - loss: 0.1183 - accuracy: 0.9585 - val_loss: 0.2535 - val_accuracy: 0.8888
Epoch 3/3
4210/4210 [==============================] - 541s 129ms/step - loss: 0.0816 - accuracy: 0.9714 - val_loss: 0.3622 - val_accuracy: 0.8784


In [15]:
# Step 5: Apply pruning (only to classification head)
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=len(tf_train_dataset) * 3
    )
}

with contextlib.nullcontext():
    # Create a new Functional model
    input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
    transformer_outputs = baseline_model.distilbert(input_ids, attention_mask=attention_mask)[0]
    cls_token = transformer_outputs[:, 0, :]  # Take [CLS] token
    # Apply pruning to dense layers only
    pre_classifier = tfmot.sparsity.keras.prune_low_magnitude(
        tf.keras.layers.Dense(768, activation='relu', name='pre_classifier'),
        **pruning_params
    )(cls_token)
    classifier = tfmot.sparsity.keras.prune_low_magnitude(
        tf.keras.layers.Dense(2, name='classifier'),
        **pruning_params
    )(pre_classifier)
    functional_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=classifier)

    # Copy weights from baseline_model to functional_model
    for layer in functional_model.layers:
        if layer.name in ['pre_classifier', 'classifier']:
            layer_wo_pruning = baseline_model.get_layer(layer.name)
            layer.set_weights(layer_wo_pruning.get_weights())

    # Compile and train the pruned model
    functional_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )
    functional_model.fit(
        tf_train_dataset,
        validation_data=tf_validation_dataset,
        epochs=3,
        callbacks=[tfmot.sparsity.keras.UpdatePruningStep()]
    )

# Strip pruning wrappers
final_pruned_model = tfmot.sparsity.keras.strip_pruning(functional_model)
final_pruned_model.save('pruned_model', save_format='tf')

Epoch 1/3
4210/4210 [==============================] - 577s 131ms/step - loss: 0.0620 - accuracy: 0.9783 - val_loss: 0.3848 - val_accuracy: 0.8979
Epoch 2/3
4210/4210 [==============================] - 543s 129ms/step - loss: 0.0502 - accuracy: 0.9821 - val_loss: 0.3744 - val_accuracy: 0.8750
Epoch 3/3
4210/4210 [==============================] - 543s 129ms/step - loss: 0.0417 - accuracy: 0.9850 - val_loss: 0.5721 - val_accuracy: 0.8739


In [13]:
!pip install tensorflow==2.15.0 transformers==4.37.0 datasets==2.15.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.4/129.4 kB 3.3 MB/s eta 0:00:00a 0:00:01
INFO: pip is looking at multiple versions of multiprocess to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 3.6 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 108.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 166.4/166.4 kB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 69.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 49.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 97.0 MB/s eta 0:00:00:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 25.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━

In [19]:
# Step 6: Knowledge Distillation
teacher_model = tf.keras.models.load_model('baseline_model')
teacher_model.trainable = False
student_model = final_pruned_model

def kd_loss(y_true, student_logits, teacher_logits, alpha=0.5, T=2):
    ce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, student_logits, from_logits=True)
    ce_loss = tf.reduce_mean(ce_loss)
    
    student_logits_soft = student_logits / T
    teacher_logits_soft = teacher_logits / T
    student_probs = tf.nn.softmax(student_logits_soft)
    teacher_probs = tf.nn.softmax(teacher_logits_soft)
    
    kl_loss = tf.keras.losses.kullback_leibler_divergence(teacher_probs, student_probs)
    kl_loss = tf.reduce_mean(kl_loss)
    
    return (1 - alpha) * ce_loss + alpha * (T ** 2) * kl_loss

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
student_model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Custom training loop for KD
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(tf_train_dataset):
        # Rename input keys to match teacher model's expected signature
        x_batch_teacher = {
            'input_ids_input_ids': x_batch['input_ids'],
            'attention_mask': x_batch['attention_mask']
        }
        with tf.GradientTape() as tape:
            student_outputs = student_model(x_batch, training=True)
            teacher_outputs = teacher_model(x_batch_teacher, training=False)
            loss = kd_loss(y_batch, student_outputs.logits if hasattr(student_outputs, 'logits') else student_outputs, teacher_outputs.logits)
        grads = tape.gradient(loss, student_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, student_model.trainable_variables))
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy()}")

kd_optimized_model = student_model
kd_optimized_model.save('kd_optimized_model', save_format='tf')

Epoch 1/3


ValueError: Exception encountered when calling layer 'tf_distil_bert_for_sequence_classification_1' (type TFDistilBertForSequenceClassification).

Could not find matching concrete function to call loaded from the SavedModel. Got:
  Positional arguments (9 total):
    * {'attention_mask': <tf.Tensor 'input_ids_1:0' shape=(16, 128) dtype=int64>,
 'input_ids_input_ids': <tf.Tensor 'input_ids:0' shape=(16, 128) dtype=int64>}
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * False
  Keyword arguments: {}

 Expected these arguments to match one of the following 2 option(s):

Option 1:
  Positional arguments (9 total):
    * {'attention_mask': TensorSpec(shape=(None, None), dtype=tf.int32, name='attention_mask'),
 'input_ids': TensorSpec(shape=(None, None), dtype=tf.int32, name='input_ids_input_ids')}
    * None
    * None
    * None
    * None
    * None
    * None
    * None
    * True
  Keyword arguments: {}

Option 2:
  Positional arguments (9 total):
    

In [21]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot
import tensorflow.lite as tflite
import time
import os
import contextlib

# Step 1: Environment setup (GPU/CPU)
strategy = None
print('Running on GPU/CPU')

# Step 2: Install required packages
!pip install tensorflow-model-optimization
!pip install ipywidgets  # For TqdmWarning, if needed

# Step 3: Load and preprocess the SST-2 dataset
dataset = load_dataset("glue", "sst2")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare tf.data.Dataset (excluding token_type_ids, ensure int32)
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
).map(lambda x, y: ({'input_ids': tf.cast(x['input_ids'], tf.int32), 'attention_mask': tf.cast(x['attention_mask'], tf.int32)}, y))

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
).map(lambda x, y: ({'input_ids': tf.cast(x['input_ids'], tf.int32), 'attention_mask': tf.cast(x['attention_mask'], tf.int32)}, y))

# Step 4: Fine-tune baseline DistilBERT
with contextlib.nullcontext():
    baseline_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
    baseline_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )
    baseline_model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

# Save baseline model in SavedModel format
baseline_model.save('baseline_model', save_format='tf')

# Step 5: Apply pruning (only to classification head)
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=len(tf_train_dataset) * 3
    )
}

with contextlib.nullcontext():
    # Create a new Functional model
    input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
    transformer_outputs = baseline_model.distilbert(input_ids, attention_mask=attention_mask)[0]
    cls_token = transformer_outputs[:, 0, :]  # Take [CLS] token
    # Apply pruning to dense layers only
    pre_classifier = tfmot.sparsity.keras.prune_low_magnitude(
        tf.keras.layers.Dense(768, activation='relu', name='pre_classifier'),
        **pruning_params
    )(cls_token)
    classifier = tfmot.sparsity.keras.prune_low_magnitude(
        tf.keras.layers.Dense(2, name='classifier'),
        **pruning_params
    )(pre_classifier)
    functional_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=classifier)

    # Copy weights from baseline_model to functional_model
    for layer in functional_model.layers:
        if layer.name in ['pre_classifier', 'classifier']:
            layer_wo_pruning = baseline_model.get_layer(layer.name)
            layer.set_weights(layer_wo_pruning.get_weights())

    # Compile and train the pruned model
    functional_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )
    functional_model.fit(
        tf_train_dataset,
        validation_data=tf_validation_dataset,
        epochs=3,
        callbacks=[tfmot.sparsity.keras.UpdatePruningStep()]
    )

# Strip pruning wrappers
final_pruned_model = tfmot.sparsity.keras.strip_pruning(functional_model)
final_pruned_model.save('pruned_model', save_format='tf')

# Step 6: Knowledge Distillation
teacher_model = tf.keras.models.load_model('baseline_model')
teacher_model.trainable = False
student_model = final_pruned_model

def kd_loss(y_true, student_logits, teacher_logits, alpha=0.5, T=2):
    ce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, student_logits, from_logits=True)
    ce_loss = tf.reduce_mean(ce_loss)
    
    student_logits_soft = student_logits / T
    teacher_logits_soft = teacher_logits / T
    student_probs = tf.nn.softmax(student_logits_soft)
    teacher_probs = tf.nn.softmax(teacher_logits_soft)
    
    kl_loss = tf.keras.losses.kullback_leibler_divergence(teacher_probs, student_probs)
    kl_loss = tf.reduce_mean(kl_loss)
    
    return (1 - alpha) * ce_loss + alpha * (T ** 2) * kl_loss

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
student_model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Custom training loop for KD
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(tf_train_dataset):
        # Cast inputs to int32 and use correct input keys
        x_batch_teacher = {
            'input_ids': tf.cast(x_batch['input_ids'], tf.int32),
            'attention_mask': tf.cast(x_batch['attention_mask'], tf.int32)
        }
        with tf.GradientTape() as tape:
            student_outputs = student_model(x_batch, training=True)
            teacher_outputs = teacher_model(x_batch_teacher, training=False)
            loss = kd_loss(y_batch, student_outputs.logits if hasattr(student_outputs, 'logits') else student_outputs, teacher_outputs.logits)
        grads = tape.gradient(loss, student_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, student_model.trainable_variables))
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy()}")

kd_optimized_model = student_model
kd_optimized_model.save('kd_optimized_model', save_format='tf')

# Step 7: Post-Training Quantization
calibration_data = tokenized_datasets["train"].select(range(100))

def representative_dataset():
    for i in range(100):
        sample = calibration_data[i]
        yield [
            np.array(sample['input_ids'], dtype=np.int32)[np.newaxis, :],
            np.array(sample['attention_mask'], dtype=np.int32)[np.newaxis, :]
        ]

converter = tf.lite.TFLiteConverter.from_saved_model('kd_optimized_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

with open('tflite_model.tflite', 'wb') as f:
    f.write(tflite_model)

# Step 8: Evaluation
def compute_accuracy(model, dataset):
    total_correct = 0
    total_samples = 0
    for x_batch, y_batch in dataset:
        # Adjust input for teacher model (TFDistilBertForSequenceClassification)
        x_batch_input = {
            'input_ids': tf.cast(x_batch['input_ids'], tf.int32),
            'attention_mask': tf.cast(x_batch['attention_mask'], tf.int32)
        } if isinstance(model, TFDistilBertForSequenceClassification) else x_batch
        outputs = model(x_batch_input, training=False)
        logits = outputs.logits if hasattr(outputs, 'logits') else outputs
        predictions = tf.argmax(logits, axis=1)
        total_correct += tf.reduce_sum(tf.cast(predictions == y_batch, tf.int32)).numpy()
        total_samples += y_batch.shape[0]
    return total_correct / total_samples

def compute_tflite_accuracy(tflite_model_path, dataset):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    total_correct = 0
    total_samples = 0
    for x_batch, y_batch in dataset:
        for i in range(x_batch['input_ids'].shape[0]):
            interpreter.set_tensor(input_details[0]['index'], tf.cast(x_batch['input_ids'][i:i+1], tf.int32))
            interpreter.set_tensor(input_details[1]['index'], tf.cast(x_batch['attention_mask'][i:i+1], tf.int32))
            interpreter.invoke()
            logits = interpreter.get_tensor(output_details[0]['index'])
            prediction = np.argmax(logits, axis=1)
            total_correct += (prediction == y_batch[i].numpy()).sum()
            total_samples += 1
    return total_correct / total_samples

def measure_inference_time(model, dataset, num_samples=100):
    start_time = time.time()
    count = 0
    for x_batch, _ in dataset:
        x_batch_input = {
            'input_ids': tf.cast(x_batch['input_ids'], tf.int32),
            'attention_mask': tf.cast(x_batch['attention_mask'], tf.int32)
        } if isinstance(model, TFDistilBertForSequenceClassification) else x_batch
        model(x_batch_input, training=False)
        count += x_batch['input_ids'].shape[0]
        if count >= num_samples:
            break
    return (time.time() - start_time) / num_samples * 1000  # ms per sample

def measure_tflite_inference_time(tflite_model_path, dataset, num_samples=100):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    start_time = time.time()
    count = 0
    for x_batch, _ in dataset:
        for i in range(x_batch['input_ids'].shape[0]):
            interpreter.set_tensor(input_details[0]['index'], tf.cast(x_batch['input_ids'][i:i+1], tf.int32))
            interpreter.set_tensor(input_details[1]['index'], tf.cast(x_batch['attention_mask'][i:i+1], tf.int32))
            interpreter.invoke()
            count += 1
            if count >= num_samples:
                break
    return (time.time() - start_time) / num_samples * 1000  # ms per sample

# Compute metrics
baseline_model = tf.keras.models.load_model('baseline_model')
pruned_model = tf.keras.models.load_model('pruned_model')
kd_optimized_model = tf.keras.models.load_model('kd_optimized_model')

baseline_accuracy = compute_accuracy(baseline_model, tf_validation_dataset)
pruned_accuracy = compute_accuracy(pruned_model, tf_validation_dataset)
kd_accuracy = compute_accuracy(kd_optimized_model, tf_validation_dataset)
tflite_accuracy = compute_tflite_accuracy('tflite_model.tflite', tf_validation_dataset)

baseline_size = sum(os.path.getsize(os.path.join('baseline_model', f)) for f in os.listdir('baseline_model') if os.path.isfile(os.path.join('baseline_model', f))) / (1024 * 1024)  # MB
pruned_size = sum(os.path.getsize(os.path.join('pruned_model', f)) for f in os.listdir('pruned_model') if os.path.isfile(os.path.join('pruned_model', f))) / (1024 * 1024)  # MB
kd_size = sum(os.path.getsize(os.path.join('kd_optimized_model', f)) for f in os.listdir('kd_optimized_model') if os.path.isfile(os.path.join('kd_optimized_model', f))) / (1024 * 1024)  # MB
tflite_size = os.path.getsize('tflite_model.tflite') / (1024 * 1024)  # MB

baseline_latency = measure_inference_time(baseline_model, tf_validation_dataset)
pruned_latency = measure_inference_time(pruned_model, tf_validation_dataset)
kd_latency = measure_inference_time(kd_optimized_model, tf_validation_dataset)
tflite_latency = measure_tflite_inference_time('tflite_model.tflite', tf_validation_dataset)

# Step 9: Plotting
metrics = {
    'Model': ['Baseline', 'Pruned', 'Distilled', 'Quantized'],
    'Accuracy': [baseline_accuracy * 100, pruned_accuracy * 100, kd_accuracy * 100, tflite_accuracy * 100],
    'Size (MB)': [baseline_size, pruned_size, kd_size, tflite_size],
    'Latency (ms)': [baseline_latency, pruned_latency, kd_latency, tflite_latency]
}

# Note: The actual plotting is handled by the ChartJS block below
plt.figure(figsize=(10, 6))
plt.subplot(1, 3, 1)
plt.bar(metrics['Model'], metrics['Accuracy'])
plt.title('Accuracy (%)')
plt.subplot(1, 3, 2)
plt.bar(metrics['Model'], metrics['Size (MB)'])
plt.title('Model Size (MB)')
plt.subplot(1, 3, 3)
plt.bar(metrics['Model'], metrics['Latency (ms)'])
plt.title('Inference Latency (ms)')
plt.tight_layout()
plt.show()

print("Metrics Table:")
print("| Model     | Accuracy (%) | Size (MB) | Latency (ms) |")
print("|-----------|--------------|-----------|--------------|")
for i in range(4):
    print(f"| {metrics['Model'][i]:<9} | {metrics['Accuracy'][i]:<12.2f} | {metrics['Size (MB)'][i]:<9.2f} | {metrics['Latency (ms)'][i]:<12.2f} |")

Running on GPU/CPU


/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with 

Epoch 1/3
4210/4210 [==============================] - 570s 130ms/step - loss: 0.2183 - accuracy: 0.9143 - val_loss: 0.2759 - val_accuracy: 0.8968
Epoch 2/3
4210/4210 [==============================] - 542s 129ms/step - loss: 0.1170 - accuracy: 0.9587 - val_loss: 0.3519 - val_accuracy: 0.8773
Epoch 3/3
4210/4210 [==============================] - 541s 129ms/step - loss: 0.0828 - accuracy: 0.9710 - val_loss: 0.2952 - val_accuracy: 0.8888
Epoch 1/3
4210/4210 [==============================] - 572s 130ms/step - loss: 0.0637 - accuracy: 0.9777 - val_loss: 0.4093 - val_accuracy: 0.8727
Epoch 2/3
4210/4210 [==============================] - 543s 129ms/step - loss: 0.0483 - accuracy: 0.9829 - val_loss: 0.4615 - val_accuracy: 0.8739
Epoch 3/3
4210/4210 [==============================] - 545s 129ms/step - loss: 0.0388 - accuracy: 0.9852 - val_loss: 0.5739 - val_accuracy: 0.8739
Epoch 1/3


AttributeError: 'dict' object has no attribute 'logits'

In [22]:
import tensorflow as tf
from transformers import DistilBertTokenizer, TFDistilBertForSequenceClassification
from datasets import load_dataset
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_model_optimization as tfmot
import tensorflow.lite as tflite
import time
import os
import contextlib

# Step 1: Environment setup (GPU/CPU)
strategy = None
print('Running on GPU/CPU')

# Step 2: Install required packages
!pip install tensorflow-model-optimization
!pip install ipywidgets  # For TqdmWarning, if needed

# Step 3: Load and preprocess the SST-2 dataset
dataset = load_dataset("glue", "sst2")
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

def tokenize_function(examples):
    return tokenizer(examples['sentence'], padding='max_length', truncation=True, max_length=128)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# Prepare tf.data.Dataset (excluding token_type_ids, ensure int32)
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

tf_train_dataset = tokenized_datasets["train"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=True,
    batch_size=16,
    collate_fn=data_collator
).map(lambda x, y: ({'input_ids': tf.cast(x['input_ids'], tf.int32), 'attention_mask': tf.cast(x['attention_mask'], tf.int32)}, y))

tf_validation_dataset = tokenized_datasets["validation"].to_tf_dataset(
    columns=["input_ids", "attention_mask"],
    label_cols=["label"],
    shuffle=False,
    batch_size=16,
    collate_fn=data_collator
).map(lambda x, y: ({'input_ids': tf.cast(x['input_ids'], tf.int32), 'attention_mask': tf.cast(x['attention_mask'], tf.int32)}, y))

# Step 4: Fine-tune baseline DistilBERT
with contextlib.nullcontext():
    baseline_model = TFDistilBertForSequenceClassification.from_pretrained('distilbert-base-uncased', num_labels=2)
    baseline_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )
    baseline_model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

# Save baseline model in SavedModel format
baseline_model.save('baseline_model', save_format='tf')

Running on GPU/CPU


/usr/local/lib/python3.11/dist-packages/datasets/arrow_dataset.py:400: FutureWarning: The output of `to_tf_dataset` will change when a passing single element list for `labels` or `columns` in the next datasets version. To return a tuple structure rather than dict, pass a single string.
Old behaviour: columns=['a'], labels=['labels'] -> (tf.Tensor, tf.Tensor)  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor)  
New behaviour: columns=['a'],labels=['labels'] -> ({'a': tf.Tensor}, {'labels': tf.Tensor})  
             : columns='a', labels='labels' -> (tf.Tensor, tf.Tensor) 
  warnings.warn(
Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFDistilBertForSequenceClassification: ['vocab_layer_norm.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.bias', 'vocab_projector.bias']
- This IS expected if you are initializing TFDistilBertForSequenceClassification from a PyTorch model trained on another task or with 

Epoch 1/3
4210/4210 [==============================] - 571s 130ms/step - loss: 0.2184 - accuracy: 0.9159 - val_loss: 0.2694 - val_accuracy: 0.9048
Epoch 2/3
4210/4210 [==============================] - 542s 129ms/step - loss: 0.1180 - accuracy: 0.9584 - val_loss: 0.3046 - val_accuracy: 0.8830
Epoch 3/3
4210/4210 [==============================] - 541s 129ms/step - loss: 0.0829 - accuracy: 0.9712 - val_loss: 0.3386 - val_accuracy: 0.8807


In [23]:
# Step 5: Apply pruning (only to classification head)
pruning_params = {
    'pruning_schedule': tfmot.sparsity.keras.PolynomialDecay(
        initial_sparsity=0.0,
        final_sparsity=0.5,
        begin_step=0,
        end_step=len(tf_train_dataset) * 3
    )
}

with contextlib.nullcontext():
    # Create a new Functional model
    input_ids = tf.keras.Input(shape=(128,), dtype=tf.int32, name="input_ids")
    attention_mask = tf.keras.Input(shape=(128,), dtype=tf.int32, name="attention_mask")
    transformer_outputs = baseline_model.distilbert(input_ids, attention_mask=attention_mask)[0]
    cls_token = transformer_outputs[:, 0, :]  # Take [CLS] token
    # Apply pruning to dense layers only
    pre_classifier = tfmot.sparsity.keras.prune_low_magnitude(
        tf.keras.layers.Dense(768, activation='relu', name='pre_classifier'),
        **pruning_params
    )(cls_token)
    classifier = tfmot.sparsity.keras.prune_low_magnitude(
        tf.keras.layers.Dense(2, name='classifier'),
        **pruning_params
    )(pre_classifier)
    functional_model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=classifier)

    # Copy weights from baseline_model to functional_model
    for layer in functional_model.layers:
        if layer.name in ['pre_classifier', 'classifier']:
            layer_wo_pruning = baseline_model.get_layer(layer.name)
            layer.set_weights(layer_wo_pruning.get_weights())

    # Compile and train the pruned model
    functional_model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
        loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
        metrics=[tf.keras.metrics.SparseCategoricalAccuracy('accuracy')]
    )
    functional_model.fit(
        tf_train_dataset,
        validation_data=tf_validation_dataset,
        epochs=3,
        callbacks=[tfmot.sparsity.keras.UpdatePruningStep()]
    )

# Strip pruning wrappers
final_pruned_model = tfmot.sparsity.keras.strip_pruning(functional_model)
final_pruned_model.save('pruned_model', save_format='tf')

Epoch 1/3
4210/4210 [==============================] - 572s 130ms/step - loss: 0.0650 - accuracy: 0.9772 - val_loss: 0.4119 - val_accuracy: 0.8819
Epoch 2/3
4210/4210 [==============================] - 543s 129ms/step - loss: 0.0491 - accuracy: 0.9834 - val_loss: 0.4500 - val_accuracy: 0.8899
Epoch 3/3
4210/4210 [==============================] - 543s 129ms/step - loss: 0.0400 - accuracy: 0.9854 - val_loss: 0.5483 - val_accuracy: 0.8796


In [24]:
# Step 6: Knowledge Distillation
teacher_model = tf.keras.models.load_model('baseline_model')
teacher_model.trainable = False
student_model = final_pruned_model

def kd_loss(y_true, student_logits, teacher_logits, alpha=0.5, T=2):
    ce_loss = tf.keras.losses.sparse_categorical_crossentropy(y_true, student_logits, from_logits=True)
    ce_loss = tf.reduce_mean(ce_loss)
    
    student_logits_soft = student_logits / T
    teacher_logits_soft = teacher_logits / T
    student_probs = tf.nn.softmax(student_logits_soft)
    teacher_probs = tf.nn.softmax(teacher_logits_soft)
    
    kl_loss = tf.keras.losses.kullback_leibler_divergence(teacher_probs, student_probs)
    kl_loss = tf.reduce_mean(kl_loss)
    
    return (1 - alpha) * ce_loss + alpha * (T ** 2) * kl_loss

optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
student_model.compile(optimizer=optimizer, loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

# Custom training loop for KD
epochs = 3
for epoch in range(epochs):
    print(f"Epoch {epoch + 1}/{epochs}")
    for step, (x_batch, y_batch) in enumerate(tf_train_dataset):
        # Cast inputs to int32 and use correct input keys
        x_batch_teacher = {
            'input_ids': tf.cast(x_batch['input_ids'], tf.int32),
            'attention_mask': tf.cast(x_batch['attention_mask'], tf.int32)
        }
        with tf.GradientTape() as tape:
            student_outputs = student_model(x_batch, training=True)  # Functional model outputs logits directly
            teacher_outputs = teacher_model(x_batch_teacher, training=False)  # Returns dict with 'logits'
            loss = kd_loss(y_batch, student_outputs, teacher_outputs['logits'])
        grads = tape.gradient(loss, student_model.trainable_variables)
        optimizer.apply_gradients(zip(grads, student_model.trainable_variables))
        if step % 100 == 0:
            print(f"Step {step}, Loss: {loss.numpy()}")

kd_optimized_model = student_model
kd_optimized_model.save('kd_optimized_model', save_format='tf')

Epoch 1/3
Step 0, Loss: 0.16732065379619598
Step 100, Loss: 0.026963941752910614
Step 200, Loss: 0.01869933307170868
Step 300, Loss: 0.027217116206884384
Step 400, Loss: 0.05904054641723633
Step 500, Loss: 0.024975813925266266
Step 600, Loss: 0.024407021701335907
Step 700, Loss: 0.06021521985530853
Step 800, Loss: 0.029428452253341675
Step 900, Loss: 0.011104256846010685
Step 1000, Loss: 0.14714956283569336
Step 1100, Loss: 0.03795892000198364
Step 1200, Loss: 0.19076687097549438
Step 1300, Loss: 0.015826400369405746
Step 1400, Loss: 0.053613483905792236
Step 1500, Loss: 0.03071068599820137
Step 1600, Loss: 0.10823647677898407
Step 1700, Loss: 0.024124812334775925
Step 1800, Loss: 0.02397768571972847
Step 1900, Loss: 0.014237001538276672
Step 2000, Loss: 0.029408998787403107
Step 2100, Loss: 0.020773548632860184
Step 2200, Loss: 0.013111310079693794
Step 2300, Loss: 0.05062306672334671
Step 2400, Loss: 0.05708516389131546
Step 2500, Loss: 0.14544281363487244
Step 2600, Loss: 0.01121774

In [25]:
# Step 7: Post-Training Quantization
calibration_data = tokenized_datasets["train"].select(range(100))

def representative_dataset():
    for i in range(100):
        sample = calibration_data[i]
        yield [
            np.array(sample['input_ids'], dtype=np.int32)[np.newaxis, :],
            np.array(sample['attention_mask'], dtype=np.int32)[np.newaxis, :]
        ]

converter = tf.lite.TFLiteConverter.from_saved_model('kd_optimized_model')
converter.optimizations = [tf.lite.Optimize.DEFAULT]
converter.representative_dataset = representative_dataset
converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
converter.inference_input_type = tf.int8
converter.inference_output_type = tf.int8
tflite_model = converter.convert()

with open('tflite_model.tflite', 'wb') as f:
    f.write(tflite_model)


W0000 00:00:1751659338.021151      35 tf_tfl_flatbuffer_helpers.cc:365] Ignored output_format.
W0000 00:00:1751659338.021202      35 tf_tfl_flatbuffer_helpers.cc:368] Ignored drop_control_dependency.
I0000 00:00:1751659338.153981      35 mlir_graph_optimization_pass.cc:401] MLIR V1 optimization pass is not enabled
fully_quantize: 0, inference_type: 6, input_inference_type: INT8, output_inference_type: INT8


In [27]:
# Step 8: Evaluation
def compute_accuracy(model, dataset):
    total_correct = 0
    total_samples = 0
    for x_batch, y_batch in dataset:
        # Adjust input for teacher model (TFDistilBertForSequenceClassification)
        x_batch_input = {
            'input_ids': tf.cast(x_batch['input_ids'], tf.int32),
            'attention_mask': tf.cast(x_batch['attention_mask'], tf.int32)
        } if isinstance(model, TFDistilBertForSequenceClassification) else x_batch
        outputs = model(x_batch_input, training=False)
        logits = outputs['logits'] if isinstance(outputs, dict) else outputs
        predictions = tf.argmax(logits, axis=1)
        total_correct += tf.reduce_sum(tf.cast(predictions == y_batch, tf.int32)).numpy()
        total_samples += y_batch.shape[0]
    return total_correct / total_samples

def compute_tflite_accuracy(tflite_model_path, dataset):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    output_details = interpreter.get_output_details()
    
    total_correct = 0
    total_samples = 0
    for x_batch, y_batch in dataset:
        for i in range(x_batch['input_ids'].shape[0]):
            interpreter.set_tensor(input_details[0]['index'], tf.cast(x_batch['input_ids'][i:i+1], tf.int32))
            interpreter.set_tensor(input_details[1]['index'], tf.cast(x_batch['attention_mask'][i:i+1], tf.int32))
            interpreter.invoke()
            logits = interpreter.get_tensor(output_details[0]['index'])
            prediction = np.argmax(logits, axis=1)
            total_correct += (prediction == y_batch[i].numpy()).sum()
            total_samples += 1
    return total_correct / total_samples

def measure_inference_time(model, dataset, num_samples=100):
    start_time = time.time()
    count = 0
    for x_batch, _ in dataset:
        x_batch_input = {
            'input_ids': tf.cast(x_batch['input_ids'], tf.int32),
            'attention_mask': tf.cast(x_batch['attention_mask'], tf.int32)
        } if isinstance(model, TFDistilBertForSequenceClassification) else x_batch
        outputs = model(x_batch_input, training=False)
        count += x_batch['input_ids'].shape[0]
        if count >= num_samples:
            break
    return (time.time() - start_time) / num_samples * 1000  # ms per sample

def measure_tflite_inference_time(tflite_model_path, dataset, num_samples=100):
    interpreter = tf.lite.Interpreter(model_path=tflite_model_path)
    interpreter.allocate_tensors()
    input_details = interpreter.get_input_details()
    start_time = time.time()
    count = 0
    for x_batch, _ in dataset:
        for i in range(x_batch['input_ids'].shape[0]):
            interpreter.set_tensor(input_details[0]['index'], tf.cast(x_batch['input_ids'][i:i+1], tf.int32))
            interpreter.set_tensor(input_details[1]['index'], tf.cast(x_batch['attention_mask'][i:i+1], tf.int32))
            interpreter.invoke()
            count += 1
            if count >= num_samples:
                break
    return (time.time() - start_time) / num_samples * 1000  # ms per sample

In [ ]:
# Compute metrics
baseline_model = tf.keras.models.load_model('baseline_model')
pruned_model = tf.keras.models.load_model('pruned_model')
kd_optimized_model = tf.keras.models.load_model('kd_optimized_model')

baseline_accuracy = compute_accuracy(baseline_model, tf_validation_dataset)
pruned_accuracy = compute_accuracy(pruned_model, tf_validation_dataset)
kd_accuracy = compute_accuracy(kd_optimized_model, tf_validation_dataset)
tflite_accuracy = compute_tflite_accuracy('tflite_model.tflite', tf_validation_dataset)

baseline_size = sum(os.path.getsize(os.path.join('baseline_model', f)) for f in os.listdir('baseline_model') if os.path.isfile(os.path.join('baseline_model', f))) / (1024 * 1024)  # MB
pruned_size = sum(os.path.getsize(os.path.join('pruned_model', f)) for f in os.listdir('pruned_model') if os.path.isfile(os.path.join('pruned_model', f))) / (1024 * 1024)  # MB
kd_size = sum(os.path.getsize(os.path.join('kd_optimized_model', f)) for f in os.listdir('kd_optimized_model') if os.path.isfile(os.path.join('kd_optimized_model', f))) / (1024 * 1024)  # MB
tflite_size = os.path.getsize('tflite_model.tflite') / (1024 * 1024)  # MB

baseline_latency = measure_inference_time(baseline_model, tf_validation_dataset)
pruned_latency = measure_inference_time(pruned_model, tf_validation_dataset)
kd_latency = measure_inference_time(kd_optimized_model, tf_validation_dataset)
tflite_latency = measure_tflite_inference_time('tflite_model.tflite', tf_validation_dataset)

# Step 9: Plotting
metrics = {
    'Model': ['Baseline', 'Pruned', 'Distilled', 'Quantized'],
    'Accuracy': [baseline_accuracy * 100, pruned_accuracy * 100, kd_accuracy * 100, tflite_accuracy * 100],
    'Size (MB)': [baseline_size, pruned_size, kd_size, tflite_size],
    'Latency (ms)': [baseline_latency, pruned_latency, kd_latency, tflite_latency]
}

# Note: The actual plotting is handled by the ChartJS block below
plt.figure(figsize=(10, 6))
plt.subplot(1, 3, 1)
plt.bar(metrics['Model'], metrics['Accuracy'])
plt.title('Accuracy (%)')
plt.subplot(1, 3, 2)
plt.bar(metrics['Model'], metrics['Size (MB)'])
plt.title('Model Size (MB)')
plt.subplot(1, 3, 3)
plt.bar(metrics['Model'], metrics['Latency (ms)'])
plt.title('Inference Latency (ms)')
plt.tight_layout()
plt.show()

print("Metrics Table:")
print("| Model     | Accuracy (%) | Size (MB) | Latency (ms) |")
print("|-----------|--------------|-----------|--------------|")
for i in range(4):
    print(f"| {metrics['Model'][i]:<9} | {metrics['Accuracy'][i]:<12.2f} | {metrics['Size (MB)'][i]:<9.2f} | {metrics['Latency (ms)'][i]:<12.2f} |")

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
